In [ ]:
from pathlib import Path
from multiprocessing import Pool
from tqdm import tqdm
from makemkv import MakeMKV, MakeMKVError
from dataclasses import dataclass, field
from datetime import datetime
from argparse import ArgumentParser
import itertools
import json
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [ ]:
# @dataclass
# class Playlist():
#     TitleNum: int = -1
#     Description: str = ''
#     FileOutput: str = ''
#     Runtime: datetime.time = ''
#     Chapters: int = -1
#     Size: int = -1

# @dataclass
# class Movie():
#     Path: Path
#     Title: str = ''
#     Size: int = -1
#     Processed: bool = False
#     Playlists: list = field(default_factory=list)

In [ ]:
class Playlist:
    def __init__(self, TitleNum=-1, SourceFileName='', Description='', FileOutput='', Runtime='', Chapters=-1, Size=-1):
        self.TitleNum = TitleNum
        self.SourceFileName = SourceFileName
        self.Description = Description
        self.FileOutput = FileOutput
        self.Runtime = Runtime
        self.Chapters = Chapters
        self.Size = Size
    
    def __repr__(self):
        return f"Playlist(TitleNum={self.TitleNum}, SourceFileName={self.SourceFileName}, Description='{self.Description}', FileOutput='{self.FileOutput}', Runtime={self.Runtime}, Chapters={self.Chapters}, Size={self.Size})"
    
    def serialize(self):
        return {
            'TitleNum': self.TitleNum,
            'SourceFileName': self.SourceFileName,
            'Description': self.Description,
            'FileOutput': self.FileOutput,
            'Runtime': str(self.Runtime),
            'Chapters': self.Chapters,
            'Size': self.Size
        }
    
    @classmethod
    def deserialize(cls, data):
        # Convert Runtime string back to datetime.time if it's a valid time string
            
        return cls(
            TitleNum=data.get('TitleNum', -1),
            SourceFileName=data.get('SourceFileName', ''),
            Description=data.get('Description', ''),
            FileOutput=data.get('FileOutput', ''),
            Runtime = data.get('Runtime', ''),    
            Chapters=data.get('Chapters', -1),
            Size=data.get('Size', -1)
        )

class Movie:
    def __init__(self, Path, Title='', Size=-1, Processed=False, Playlists=None):
        self.Path = Path
        self.Title = Title
        self.Size = Size
        self.Processed = Processed
        self.Playlists = Playlists if Playlists is not None else []
    
    def __repr__(self):
        return f"Movie(Path='{self.Path}', Title='{self.Title}', Size={self.Size}, Processed={self.Processed}, Playlists={self.Playlists})"
    
    def serialize(self):
        return {
            'Path': str(self.Path),
            'Title': self.Title,
            'Size': self.Size,
            'Processed': self.Processed,
            'Playlists': [playlist.serialize() for playlist in self.Playlists]
        }
    
    @classmethod
    def deserialize(cls, data):
        # Deserialize nested Playlist objects
        playlists = [
            Playlist.deserialize(playlist_data)
            for playlist_data in data.get('Playlists', [])
        ]
        
        return cls(
            Path=data.get('Path', ''),
            Title=data.get('Title', ''),
            Size=data.get('Size', -1),
            Processed=data.get('Processed', False),
            Playlists=playlists
        )

In [ ]:
def findMovieFiles(root: Path) -> list:
    movie_isos = [x for x in root.glob('**/*') if x.suffix == ".iso" or x.suffix == ".ISO"]
    # print(len(movie_isos))
    return movie_isos

def mkvProgress(task_description, progress, max):
    # print(f'{task_description}\t{progress}/{max}')
    # for progress in tqdm(range(max), desc=task_description):
    #     continue
    pass

def getBasicMovieFileDetails(file: Path) -> Movie:
    return Movie(Path=file, Size=file.stat().st_size)

def getMovieDetails(movie: Movie, min_length: int = 1200, size_cutoff: float = 0.5) -> Movie:
    print(f'Processing: {movie.Path}')
    try:
        mkv = MakeMKV(input=movie.Path, minlength=min_length)
        info = mkv.info()
    except MakeMKVError:
        print(f'ERROR, problem with {movie}.')
        pass
    else:
        for idx, i in enumerate(info['titles']):
            if i.get('chapter_count') is None:
                continue
            if i['size'] > (movie.Size * size_cutoff):
                movie.Title = i['name'] if i.get('name') is not None else ''
                movie.Playlists.append(Playlist(TitleNum=idx, SourceFileName=i['source_filename'], Description=i['information'], FileOutput=i['file_output'], Runtime=i['length'], Chapters=i['chapter_count'], Size=i['size']))
    return movie

def remuxMovie(movie: Movie, dest_root: Path, min_length:int=1200):
    movie_dir = Path(movie.Path).parent
    folder_name = movie_dir.name
    dest_path: Path = dest_root / folder_name

    if not dest_path.exists():
        Path.mkdir(dest_path)
    
    try:
        mkv = MakeMKV(input=movie.Path, minlength=min_length)
        mkv.mkv(movie.Playlists[0].TitleNum, dest_path)
    except KeyboardInterrupt:
        mkv.kill()
    finally:
        print('MakeMKV process killed.')

def remuxMovieList(movies: list[Movie], dest_root: Path, min_length: int=1200):
    num_movies = len(movies)
    
    try:
        for m in tqdm(movies):
            if not m.Processed:
                print(f'Working on movie {m.Title}')
                remuxMovie(m, dest_root, min_length)
                m.Processed = True
    except KeyboardInterrupt:
        dumpMovieList(movies, 'in_process.json')
    finally:
        print(f'Program interrupted. Load in_process.json to continue.')

def dumpMovieList(movies: list, file: Path):
    with open(file, 'w') as f:
        json.dump([movie.serialize() for movie in movies], fp=f, indent=2)
    print('Saved list.')

def loadMovieList(file: Path) -> list[Movie]:
    with open(file, 'r') as f:
        movie_list = [m for m in json.load(f)]
    return [Movie.deserialize(movie_data) for movie_data in movie_list]

In [ ]:
movies_e_details = list(map(lambda m: getBasicMovieFileDetails(m), findMovieFiles(Path('/mnt/e/Blu-ray/Movies'))))
movies_f_details = list(map(lambda m: getBasicMovieFileDetails(m), findMovieFiles(Path('/mnt/f/Blu-ray/Movies'))))
movies_g_details = list(map(lambda m: getBasicMovieFileDetails(m), findMovieFiles(Path('/mnt/g/Blu-ray/Movies'))))
all_movie_details = [movies_e_details, movies_f_details, movies_g_details]

In [ ]:
for l in all_movie_details:
    with Pool(4) as p:
        result = p.map(getMovieDetails, l)

In [ ]:
has_mult_playlists = [m for m in result if len(m.Playlists) > 1]
# dumpMovieList(result, 'movies_f.json')

In [10]:
# e = loadMovieList('movies.json')
# f = loadMovieList('movies_f.json')
# g = loadMovieList('movies_g.json')
all_movies = loadMovieList('all_movies.json')

In [11]:
import operator
original_size = sum([m.Size for m in all_movies if not m.Processed])
mkv_size = sum([m.Playlists[0].Size for m in all_movies if not m.Processed])
print(f'Current Size: {original_size / (1024**4)}\nResult Size: {mkv_size / (1024**4)}\nDifference: {(original_size - mkv_size) / (1024**4)}')

Current Size: 21.650456953413595
Result Size: 17.3176779811738
Difference: 4.332778972239794


In [ ]:

dumpMovieList(all_movies, 'all_movies.josn')

In [ ]:
remuxMovieList(all_movies, Path('truenas/Movies'))